# Week 4

During week 4 of the project, we worked on adding the sound effects. In order to do so, we first used MATLAB to convert some `wav` audio files into a large array and then used that array to send data directly to the DAC using DMA. 

---

## The Design

#### 1. Converting audio files to arrays

In order to add the audio in game, we first had to covert it into a form which can directly be read by the microcontroller. Moreover, it had to be in a format that can be directly pushed to the DAC. Hence, we also had to append the 4 DAC configuration bits in the MSB of each element of the audio array. To do so, we used a MATLAB script which can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/ConvertAudioToArray.txt).

First, we defined the upper and lower bounds of the audio file in terms of the audio sample number. This was done to remove any unwanted sections of the audio. Next, we defined the `FILE_NAME` and the `TEXT_FILE`. These are the audio input file name and the output text file name respectively. Lastly, we defined the `BitRate` which is the new sampling frequency of the audio. This was done to reduce the size of the audio array. _Note: Reducing the bitrate of the audio also reduces the audio quality._

```c
UPPER_RANGE = %Your Upper Range%
LOWER_RANGE = %Your Upper Range%
FILE_NAME = %Audio File Name%
TEXT_FILE = %Text File Name%
BitRate = 8000
Range = [LOWER_RANGE, UPPER_RANGE];
```

<br>

Next we used the `audioread()` function to read the audio file into an array `y` and get its default sampling frequency `Fs`. In order to change the sampling frequency, we used the `resample()` function to change its bitrate. _Note: This function requires the DSP toolbox to be installed in MATLAB to work properly._ The audio file we read was a 2 channel audio, hence we sliced it down to a single channel and stored it in `ys` as a column matrix. The amplitude of the audio input stored in `ys` ranges from -1 to 1. However, we have a 12-bit DAC. Therefore, we had to rescale the amplitude from 0 to 4095. This was done using the `rescale()` function. Laslty, we force casted each element of the audio to a `uint16` data type in order to make it compatible with the DAC.

```c
[y, Fs] = audioread(FILE_NAME, Range);
yn = resample(y, BitRate, Fs);
ys = yn(:, 1);
yqfinal = rescale(ys, 0, 4095);
ysfinal = cast(yqfinal, 'uint16');
```

<br>

```c
file = fopen(TEXT_FILE, 'w');
for i = 1 : length(ysfinal)
    yfinal(i) = bitor(ysfinal(i), 0b0011000000000000);
    fprintf(file, "%d, ", yfinal(i));
end
fclose(file);
```